# Capstone Project: ResNet-50 for Cats.Vs.Dogs

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import keras

In [ ]:
from __future__ import division
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input, Lambda, Reshape
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD, Nadam
from keras.utils.data_utils import get_file

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import random
from collections import Counter
import tensorflow as tf
from utils import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SEED = 42
np.random.seed(SEED)

In [ ]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True 
set_session(tf.Session(config=config))

In [ ]:
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

GPUs = get_available_gpus()

In [ ]:
print(GPUs)

In [ ]:
DATA_DIR = '/home/Drive2/rishabh/'
TRAIN_FEATURES = os.path.join(DATA_DIR, 'bottleneck_features_train_activation_46.npy')
TEST_FEATURES = os.path.join(DATA_DIR, 'bottleneck_features_test_activation_46.npy')
CHECKPOINTED_WEIGHTS = os.path.join(DATA_DIR, 'checkpointed_weights_siamese.hdf5')
INIT_WEIGHTS = os.path.join(DATA_DIR, 'init_weights_base_siamese.hdf5')
MODEL_IMAGE = os.path.join(DATA_DIR, 'resnet50.png')

## Data preprocessing

- The images in train folder are divided into a training set and a validation set.
- The images both in training set and validation set are separately divided into two folders -- cat and dog according to their lables.

*(the two steps above were finished in  Preprocessing train dataset.ipynb)*

- The RGB color values of the images are rescaled to 0~1.
- The size of the images are resized to 224*224.


In [ ]:
image_width = 224
image_height = 224
image_size = (image_width, image_height)
BATCH_SIZE = 2000

train_datagen = ImageDataGenerator(rescale=1.0/255)#, 
#                 zca_whitening=True, zca_epsilon=1e-5)

train_generator = train_datagen.flow_from_directory(
        'mytrain',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=BATCH_SIZE,
        class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
        'myvalid',  # this is the target directory
        target_size=image_size,  # all images will be resized to 224x224
        batch_size=BATCH_SIZE,
        class_mode='binary')


In [ ]:
# x, y = train_generator.next()

# plt.figure(figsize=(16, 8))
# for i, (img, label) in enumerate(zip(x, y)):
#     if i >= 18:
#         break
#     plt.subplot(3, 6, i+1)
#     if label == 1:
#         plt.title('dog')
#     else:
#         plt.title('cat')
#     plt.axis('off')
#     plt.imshow(img, interpolation="nearest")

# # Delete the dataset generated above
# del x, y

In [ ]:
X_train, y_train = train_generator.next()
X_test, y_test = test_generator.next()

In [ ]:
print(X_train.shape)

## Build the structure of ResNet-50 for Cats.Vs.Dogs

1. Build the structure of ResNet-50 without top layer.
2. Add top layer to ResNet-50.
3. Setup training attribute.
4. Compile the model.

### 1.Build the structure of ResNet-50 without top layer. 
Pass the train and test data throught the network and del the model from memory

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean((1 - y_true) * K.square(y_pred) +
                  y_true * K.square(K.maximum(margin - y_pred, 0)))

In [ ]:
from keras.applications.resnet50 import ResNet50
size = (image_width, image_height, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=size)

In [ ]:
from IPython.display import Image
base_model.summary()
Image(filename=MODEL_IMAGE) 

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers.pooling import AveragePooling2D

def create_train_test_features():
    with tf.device(GPUs[0]):
        x = base_model.get_layer('activation_46').output
        x = AveragePooling2D((7, 7), name='avg_pool')(x)
        flatten = Flatten()(x)
        model = Model(inputs = base_model.input, outputs = flatten)

    #   Train data
    bottleneck_features_train = model.predict(X_train)
    # save the output as a Numpy array
    np.save(open(TRAIN_FEATURES, 'w'), bottleneck_features_train)

    # Test data
    bottleneck_features_test = model.predict(X_test)
    # save the output as a Numpy array
    np.save(open(TEST_FEATURES, 'w'), bottleneck_features_test)
    del model
    
# if not os.path.exists(TRAIN_FEATURES):
    create_train_test_features()
del base_model


### Build the Model

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

INPUT_SHAPE = 2048
reg = 1e-4

GPU = GPUs[0]
with tf.device(GPU):
    base_network = Sequential()
    base_network.add(Dense(8094, input_shape=(INPUT_SHAPE,), kernel_regularizer = l2(reg)))
    base_network.add(BatchNormalization())
    base_network.add(Activation('relu'))
    base_network.add(Dense(4096, input_shape=(INPUT_SHAPE,), kernel_regularizer = l2(reg)))
    base_network.add(BatchNormalization())
    base_network.add(Activation('relu'))
    base_network.add(Dense(4096, kernel_regularizer = l2(reg)))
    base_network.add(BatchNormalization())
    base_network.add(Activation('relu'))
    base_network.add(Dense(1024, kernel_regularizer = l2(reg), activation='tanh'))
    base_network.summary()

### Siamese Net

In [ ]:
print(GPUs)

In [ ]:
from keras import layers

with tf.device(GPU):
    input_a = Input(shape=(INPUT_SHAPE,))
    processed_a = base_network(input_a)
    input_b = Input(shape=(INPUT_SHAPE,))
    processed_b = base_network(input_b)
    cos_distance = layers.Dot(axes = -1, normalize = True)([processed_a, processed_b])
    siamese_net = Model([input_a, input_b], cos_distance)
siamese_net.save_weights(INIT_WEIGHTS)

In [ ]:
siamese_net.summary()


## Train ResNet-50 for Cats.Vs.Dogs and Save the best model.

### 7. Compile the model.

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
              patience=5, verbose = 1, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='train_loss',
                              min_delta=1e-4,
                              patience=15,
                              verbose=0, mode='auto')
checkpointer = ModelCheckpoint(filepath=CHECKPOINTED_WEIGHTS, verbose=1, save_best_only=True, period=5)

In [ ]:
NUM_TRAIN_PAIRS = 1000
NUM_VAL_PAIRS = 20000
BATCH_SIZE = 128

train_data = np.load(TRAIN_FEATURES)
datagen = DataGenerator(train_data, y_train, batch_sz = BATCH_SIZE, num_train_pairs = NUM_TRAIN_PAIRS, 
                        num_val_pairs = NUM_VAL_PAIRS, verbose = True)

In [ ]:
nadam = Nadam(lr=1e-2)
siamese_net.compile(optimizer=nadam, loss='binary_crossentropy', metrics=['accuracy'])
siamese_net.load_weights(INIT_WEIGHTS)

In [ ]:
STEPS_PER_EPOCH = NUM_TRAIN_PAIRS//BATCH_SIZE
VALIDATION_STEPS = NUM_VAL_PAIRS//BATCH_SIZE

# for lr in np.logspace(-5, 0, 20):
#     nadam = Nadam(lr=lr)
#     siamese_net.compile(optimizer=nadam, loss='binary_crossentropy')
#     siamese_net.load_weights(INIT_WEIGHTS)
history = siamese_net.fit_generator(
        datagen.next_train(),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=250,
        validation_data=datagen.next_val(),
        validation_steps=VALIDATION_STEPS,
        callbacks = [reduce_lr, checkpointer])
#     losses = history.history
#     print("lr:{} loss:{} val_loss:{}".format(lr, losses['loss'][-1], losses['val_loss'][-1]))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# import h5py
# siamese_net.save_weights('saved_weights_model.h5', overwrite=True)
# del model

In [ ]:
siamese_net.load_weights('checkpointed_weights.hdf5')
test_data = np.load(open('bottleneck_features_test.npy'))

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
    
def kernel(x, y):
    return siamese_net.predict([x, y])[:, 0]

def compute_kernel(X, Y):
    n1, n2 = X.shape[0], Y.shape[0]
    columns = [np.array([x] * n2) for x in X]    
    dot_products =[kernel(col, Y) for col in columns]
    return np.vstack(dot_products)

In [ ]:
%%time
n_samples = 2000
train_examples = train_data[0: n_samples]
train_kernel = compute_kernel(train_examples, train_examples)

In [ ]:
y_train_true = y_train[: n_samples]
C = [0.001,0.02,0.04,0.05,0.06,0.07, 0.08, 0.1, 0.2, 0.5,0.6, 0.7,0.8, 0.9, 1.0, 2.0, 5.0, 7.0, 10.0,40.0,100.0]
# print(max_C)
max_acc = 0
for slack in C: 
    clf = svm.SVC(C = slack, kernel='precomputed')
    clf.fit(train_kernel, y_train_true)
    y_train_pred = clf.predict(train_kernel)
    acc = accuracy_score(y_train_true, y_train_pred)
    if acc > max_acc:
        max_acc = acc
        max_C = slack
# print(confusion_matrix(y_train_true, y_train_pred))

In [ ]:
print(max_acc, max_C)

In [ ]:
%%time
n = 1000
test_kernel = compute_kernel(test_data[:n], train_examples)
y_pred = clf.predict(test_kernel)

In [ ]:
y_true = y_test[:n]

In [ ]:
print(accuracy_score(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))